# Help Desk Scheduling with PuLP (Colab Ready)

This notebook shows how to experiment with the standalone `scheduler_lp` module in a cloud environment such as Google Colab.
It downloads the latest module files directly from GitHub, installs the required solver dependency, and
walks through building a toy rostering instance that you can adapt for your own data.

## 1. Environment setup
Run the next two cells once per session to install dependencies and fetch the module files.
Feel free to replace the GitHub branch or file URLs if you want to test local changes.

In [ ]:
# Install PuLP (quiet mode keeps the log short).
%pip install --quiet pulp==2.7.0

In [ ]:
"""Download the scheduler_lp module from GitHub so we can import it locally."""
from pathlib import Path
from urllib.request import urlopen

RAW_BASE = "https://raw.githubusercontent.com/firepenguindisopanda/INFO3604-help-desk-rostering/routes_v2_fix/scheduler_lp"
FILES = {
    "__init__.py": f"{RAW_BASE}/__init__.py",
    "linear_scheduler.py": f"{RAW_BASE}/linear_scheduler.py",
    "examples.py": f"{RAW_BASE}/examples.py",
}

target_dir = Path("scheduler_lp")
target_dir.mkdir(parents=True, exist_ok=True)

for name, url in FILES.items():
    with urlopen(url) as response:
        data = response.read()
    (target_dir / name).write_bytes(data)

print(f"Downloaded {len(FILES)} files into {target_dir.resolve()}")

## 2. Build a demo rostering instance
We will import the module, create a few assistants with availability windows, set up shifts with course demand,
and then run the solver.

All classes are plain dataclasses, so you can generate them from CSV/JSON or any custom loader.

In [ ]:
from datetime import time

from scheduler_lp import (
    AvailabilityWindow,
    Assistant,
    CourseDemand,
    SchedulerConfig,
    Shift,
    solve_helpdesk_schedule,
)

In [ ]:
# Define a small pool of assistants (feel free to tweak these).
assistants = [
    Assistant(
        id="alice",
        courses=["COMP1600", "COMP2603"],
        availability=[
            AvailabilityWindow(day_of_week=0, start=time(9), end=time(17)),
            AvailabilityWindow(day_of_week=2, start=time(9), end=time(17)),
        ],
        min_hours=4,
        max_hours=8,
    ),
    Assistant(
        id="bryan",
        courses=["COMP1600", "INFO2602"],
        availability=[
            AvailabilityWindow(day_of_week=0, start=time(12), end=time(17)),
            AvailabilityWindow(day_of_week=1, start=time(9), end=time(17)),
        ],
        min_hours=4,
        max_hours=8,
    ),
    Assistant(
        id="carmen",
        courses=["COMP2603", "INFO2602"],
        availability=[
            AvailabilityWindow(day_of_week=1, start=time(9), end=time(13)),
            AvailabilityWindow(day_of_week=3, start=time(9), end=time(17)),
        ],
        min_hours=4,
    ),
]

# Define shifts with course demand (hour-long slots Monday-Wednesday).
shifts = [
    Shift(
        id="mon_09",
        day_of_week=0,
        start=time(9),
        end=time(10),
        course_demands=[
            CourseDemand(course_code="COMP1600", tutors_required=1, weight=2.0),
            CourseDemand(course_code="COMP2603", tutors_required=1, weight=1.0),
        ],
    ),
    Shift(
        id="mon_10",
        day_of_week=0,
        start=time(10),
        end=time(11),
        course_demands=[
            CourseDemand(course_code="COMP1600", tutors_required=1, weight=2.0),
            CourseDemand(course_code="INFO2602", tutors_required=1, weight=3.0),
        ],
    ),
    Shift(
        id="tue_09",
        day_of_week=1,
        start=time(9),
        end=time(10),
        course_demands=[
            CourseDemand(course_code="COMP1600", tutors_required=1, weight=1.0),
            CourseDemand(course_code="INFO2602", tutors_required=1, weight=2.0),
        ],
    ),
    Shift(
        id="wed_13",
        day_of_week=2,
        start=time(13),
        end=time(14),
        course_demands=[
            CourseDemand(course_code="COMP2603", tutors_required=1, weight=1.0),
            CourseDemand(course_code="INFO2602", tutors_required=1, weight=2.5),
        ],
    ),
]

config = SchedulerConfig(
    course_shortfall_penalty=5.0,
    understaffed_penalty=40.0,
    min_hours_penalty=12.0,
    max_hours_penalty=5.0,
    solver_time_limit=60,
    log_solver_output=False,
)

In [ ]:
result = solve_helpdesk_schedule(assistants, shifts, config=config)

print(f"Solver status: {result.status}")
print(f"Objective value: {result.objective_value}")
print(
for assistant_id, shift_id in result.assignments:
    print(f"  - {assistant_id} -> {shift_id}")

print(
for assistant_id, hours in result.assistant_hours.items():
    print(f"  - {assistant_id}: {hours} hours")

print(
for key, value in result.course_shortfalls.items():
    print(f"  - {key}: {value}")

print(
for shift_id, value in result.staff_shortfalls.items():
    print(f"  - {shift_id}: {value}")

## 3. Next steps
- Replace the hard-coded lists with data loaded from CSV/Google Sheets.
- Adjust penalty weights in `SchedulerConfig` to reflect your policies.
- Inspect `result.to_assignment_matrix()` for easier downstream formatting.
- Use the downloaded `examples.py` for more ideas ("Run Module" > `python -m scheduler_lp.examples`).